In [4]:
import jax.numpy as jnp
import jax


def f(x):
    return jnp.sum(x)


expr = jax.make_jaxpr(f)(jnp.array([1.0, 1.0, 2.0]))
# print(expr)
print(expr)

# print(expr.eqns[0].primitive)
# print(expr.eqns[0].params)
print(expr.eqns[0].invars[0].count)
print(expr.eqns[1].invars[1].count)
print(type(expr.eqns[2].invars[1].val))
print(type(expr.eqns[2].invars[1].aval))

{ lambda ; a:f32[3]. let b:f32[] = reduce_sum[axes=(0,)] a in (b,) }
21


IndexError: list index out of range